In [14]:
import os
import json
import glob
import numpy as np
import scipy.io.wavfile
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
import torchaudio
import sys
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from IPython.display import Audio, display

# Creación y guardado de espectrogramas 
La función plot_specgram crea espectrogramas a partir de:
    
    -waveform: tensor de torachaudio
    -sample_rate: entero
    -title: str para el nombre del archivo
    -save: boolean para guardar o no el archivo, por defecto true
    -xlim: real para limitar el dominio (eje temporal), por defecto sin limitaciones
    
Además guarda el resultado en formato png en la ruta "./nsynth-test/spectrograms/". Elimina los ejes y la información de los ejes, para quedarse exclusivamente con la imagen.
#### Produce imágenes en formato png de 352x235 pixeles

### Consideraciones
Dado que se pierde la referencia de la duración de la muestra, a la hora de ponerlo en práctica con muestras de más de 4 segundos, sería necesario dividirlas en fracciones de 4 segundos para estudiarlas.

### PROBLEMA!!!
Para los instrumentos capaces de producir exactamente cero ruido (generalmente los sintéticos o aquellos en los que la muestra dura menos de 4 segundos), produce franjas blancas en el intervalo de silencio. Esto no es muy realista sabiendo que las muestras reales tendrían ruido ambiente.


In [15]:
def plot_specgram(waveform, sample_rate, title, save=True, xlim=None):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].specgram(waveform[c], Fs=sample_rate)
        if num_channels > 1:
            axes[c].set_ylabel(f'Channel {c+1}')
        if xlim:
            axes[c].set_xlim(xlim)
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.savefig("./nsynth-test/spectrograms/"+title+".png", bbox_inches='tight')
    plt.close()

# Transformación de .wav a .png
Usando la funcion anterior transforma el contenido de "./nsynth-test/audio/", registrado en "nsynth-test/examples.json", y crea un diccionario que mapea el nombre SIN EXTENSION del archivo y la familia de instrumentos a la que pertenece.

In [16]:
with open("nsynth-test/examples.json", "r") as f:
    datos = json.loads(f.read())

leyenda = {}
for dato in datos:
    waveform, sample_rate = torchaudio.load("./nsynth-test/audio/"+dato+".wav")
    plot_specgram(waveform, sample_rate, dato)
    leyenda[dato] = datos[dato]["instrument_family_str"]


In [18]:
f = open('leyenda.json', 'w')
f.write(json.dumps(leyenda))
f.close()

# Revisar muestras como bass_synthtic_068-107-100

In [ ]:
os.listdir("./nsynth-test/audio/")

In [4]:
waveform, sample_rate = torchaudio.load("./nsynth-test/audio/bass_synthtic_068-107-100.wav")

In [ ]:
plot_specgram(waveform, sample_rate, title ="bass_synthtic_068-107-100", save= False )